<a href="https://www.nvidia.com/dli"><img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/></a>

## 评估：构建实时视频 AI 应用 ##
在此 Notebook 中，您将利用在本课程中学到的内容完成评估。评估分为几个步骤，您需在各个步骤中生成用于评级的文本文件。评级的评量标准如下：
<table border="1" class="dataframe" align='left'>  
<thead>    
<tr style="text-align: left;">      <th>第n步</th>      <th>有几个 &lt;FIXME&gt;</th>      <th>分数</th>    
</tr>  
</thead>
    
<tbody>    <tr>      <td>0. 问题描述</td>      <td>0</td>      <td>0</td>    </tr>    <tr>      <td>1. 了解输入视频</td>      <td>5</td>      <td>10</td>    </tr>    <tr>      <td>2. 就 AI 推理开展头脑风暴并下载预训练模型</td>      <td>2</td>      <td>10</td>    </tr>    <tr>      <td>3. 编辑推理配置文件</td>      <td>10</td>      <td>10</td>    </tr>    <tr>      <td>4. 构建并运行 DeepStream 工作流</td>      <td>20</td>      <td>20</td>    </tr>    <tr>      <td>5. 分析结果</td>      <td>1</td>      <td>10</td>    </tr>    <tr>      <td>额外练习。可视化帧</td>      <td>0</td>      <td>0</td>    </tr>  
</tbody>
    
</table>
<br>
<p><img src='images/iva_framework.png' width=600></p>
</br>

### 第 0 步：问题描述 ###
您是车队管理公司的开发人员。最近公司在所有车辆上安装了行车记录仪，并准备实施 AI 来分析车队的驾驶行为。您注意到车队存在一个问题，当车辆在另一车辆后方行驶且未留够避免碰撞的停车间距时，如果前方车辆突然停车，就会造成[追尾](https://en.wikipedia.org/wiki/Tailgating)。因此您决定构建 DeepStream 应用来帮助监控此行为。在这种情况下，您希望能够记录发生追尾的次数，以便了解发生的频率。请注意，输入视频源是静态文件，但您可以轻松修改工作流，从而使用实时视频。

**说明**：<br>
0.1 执行此单元以将目标视频设置为环境变量。<br>

In [ ]:
# 0.1
# DO NOT CHANGE THIS CELL
import os
os.environ['TARGET_VIDEO_PATH']='data/assessment_stream.mp4'

### 第 1 步：了解输入视频 ###
第一步是先了解输入视频的属性，然后再设计一个系统来处理这些视频。

使用 `ffprobe`（[按需参阅文档](https://ffmpeg.org/ffprobe.html)）函数获取输入视频的 `height`、`width` 和 `frame rate`。此外，我们还会使用 `-hide_banner` 选项充分减少文本输出。

**说明**：<br>
1.1 执行此单元以预览视频。<br>
1.2 执行此单元从输入视频流中收集信息。<br>
1.3 *只将* `<FIXME>` 修改为正确的值，并执行此单元以标记您的答案。_您可以多次执行此单元，直到满意为止_ 。<br>

In [ ]:
# 1.1
# DO NOT CHANGE THIS CELL
from IPython.display import Video
Video(os.environ['TARGET_VIDEO_PATH'], width=720)

In [ ]:
# 1.2
# DO NOT CHANGE THIS CELL
!ffprobe -i $TARGET_VIDEO_PATH \
         -hide_banner \
         2>&1| tee my_assessment/video_profile.txt 

In [ ]:
# 1.3
FRAME_RATE=<<<<FIXME>>>>
FRAME_HEIGHT=<<<<FIXME>>>>
FRAME_WIDTH=<<<<FIXME>>>>
FRAME_CODEC='<<<<FIXME>>>>'
FRAME_COLOR_FORMAT='<<<<FIXME>>>>'

# DO NOT CHANGE BELOW
Answer=f"""\
FRAME RATE: {round(FRAME_RATE)} FPS \
HEIGHT: {FRAME_HEIGHT} \
WIDTH: {FRAME_WIDTH} \
FRAME_CODEC: {FRAME_CODEC} \
FRAME_COLOR_FORMAT: {FRAME_COLOR_FORMAT} \
"""

!echo $Answer > my_assessment/answer_1.txt

### 第 2 步：就 AI 推理开展头脑风暴并下载预训练模型 ###
下一步是就实现目标所需的 AI 推理开展头脑风暴。对于此应用，我们需要检测视频帧中的汽车，并识别边界框低于阈值的情况（如下所示）。

<p><img src='images/tailgating_logic.png' width=720></p>

幸运的是，[DashCamNet](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/tao/models/dashcamnet) 专用物体检测模型所用的训练数据与我们的视频相似。我们可以使用 [NGC CLI](https://ngc.nvidia.com/setup/installers/cli) 下载适合我们应用的 [DashCamNet](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/tao/models/dashcamnet) 模型

**说明**：<br>
2.1 执行此单元以安装 NGC CLI。<br>
2.2 执行此单元以使用可列出所有可用模型的 `ngc registry mode list` 命令。使用 `--column name`、`--column repository` 和 `--column application` 选项，以仅显示相关列。之后，查看 [DashCamNet](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/tao/models/dashcamnet) 的模型卡，并确认此模型是否适用。<br>
2.3 *只将* `<FIXME>` 修改为正确的值，并执行此单元以下载 [DashCamNet](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/tao/models/dashcamnet) 模型。该命令的输出将生成一个文本文件，以供评级使用。_您可以多次执行此单元，直到满意为止_ 。<br>

In [2]:
# 2.1
# DO NOT CHANGE THIS CELL
import os
os.environ['NGC_DIR']='/dli/task/ngc_assets'

%env CLI=ngccli_cat_linux.zip
!mkdir -p $NGC_DIR/ngccli
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $NGC_DIR/ngccli
!unzip -u "$NGC_DIR/ngccli/$CLI" \
       -d $NGC_DIR/ngccli/
!rm $NGC_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("NGC_DIR", ""), os.getenv("PATH", ""))

In [ ]:
# 2.2
# DO NOT CHANGE THIS CELL
!ngc registry model list nvidia/tao/* --column name --column repository --column application

In [ ]:
# 2.3
!ngc registry model <<<<FIXME>>>> nvidia/tao/<<<<FIXME>>>>:pruned_v1.0 --dest $NGC_DIR \
2>&1| tee my_assessment/answer_2.txt

### 第 3 步：编辑推理配置文件 ###
下一步是修改用于配置 AI 推理插件的 Gst-nvinfer 配置文件。为此可以手动创建新文本文件，既可以从头开始创建文件，也可以使用[提供的模板](spec_files/pgie_config_dashcamnet.txt)。此外，还可以点击[此处](https://github.com/NVIDIA-AI-IOT/deepstream_python_apps)，参考示例应用和配置文件。创建配置文件时，要注意的字段如下：

使用从 NGC 下载的 TAO 工具包模型时，需使用以下属性：
* `tlt-encoded-model` - TAO 工具包编码模型的路径名。
* `tlt-model-key` - TAO 工具包编码模型的模型加载密钥。
* `labelfile-path` - 包含模型标签的文本文件的路径名。
* `int8-calib-file` - INT8 校正文件的路径名，该文件可用于调整 FP32 模型的动态范围（仅支持 INT8）。
* `uff-input-blob-name` - UFF 文件中输入 Blob 的名称。
* `output-blob-names` - 输出层名称数组。
* `input-dims` - 模型的维度，即[通道、高度、宽度、输入顺序]，如果输入顺序为 0，即为 NCHW。
* `net-scale-factor` - 像素归一化因子_（默认值为 1）_。

推荐属性：
* `batch-size` - 一同接受批量推理操作的帧数_（默认值为 1）_。

检测器的必要属性：
* `num-detected-classes` - 网络检测到的类数量。

检测器的可选属性：
* `cluster-mode` - 使用的集群算法_（默认值为 0，即矩形分组）_。
* `interval` - 跳过推理的连续批数_（仅针对主要模式，| 默认值为 0）_。

其他可选属性：
* `network-mode` - 用于推理的数据格式_（0 表示 FP32，1 表示 INT8，2 表示 FP16 模式，| 默认值为 0，即 FP32）_。
* `process-mode` - 插件的运行模式_（主要或次要）__（默认值为 1，即主要模式）_。
* `model-color-format` - 模型所需的色彩格式_（默认值为 0，即 RGB）_。
* `gie-unique-id` - 要分配给 GIE 的唯一 ID，用于帮助应用和其他元件识别检测到的边界框和标签_（默认值为 0）_。
* `model-engine-file` - 序列化模型引擎文件的路径名。
* `gpu-id` - 执行预处理/推理所用 GPU 的设备 ID_（仅限 dGPU）_。

**注意**：配置文件中的值会被通过 GObject 属性设置的值覆盖。务必谨记的另一点是：推荐的属性为主要检测器的专属属性，您需要为辅助设备或分类器配置其他属性。您可以在[模型卡](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/tao/models/dashcamnet)上找到所需的大部分信息：

<p><img src='images/model_card.png' width=720></p>

**说明**：
<br>
3.1. 打开并查看[配置文件](spec_files/pgie_config_dashcamnet.txt)。<br>
3.2. *只将* 配置文件中 `<FIXME>` 更新为正确的值，并**保存更改**。之后，确保在此单元中引用配置文件的正确路径，并执行此单元以标记您的答案。_您可以多次执行此单元，直到满意为止_ 。<br>

In [ ]:
# 3.2
os.environ['SPEC_FILE']='/dli/task/spec_files/pgie_config_dashcamnet.txt'

# DO NOT CHANGE BELOW
!cat $SPEC_FILE
!cp -v $SPEC_FILE my_assessment/answer_3.txt

### 第 4 步：构建并运行 DeepStream 工作流 ###
接下来，需要构建工作流。向特定函数添加工作流创建和启动过程，以便轻松调用。在运行工作流之前，还需要实施探针回调函数。我们已提供功能架构和框架，以供此应用使用。此工作流的体系架构如下。

<p><img src='images/assessment_pipeline.png' width=1080></p>

确定车辆是否追尾的逻辑基于下图所示的检测物体边界框的坐标：

<p><img src='images/tailgate_metrics.png' width=720></p>

将探针附加至 _nvdsosd_ 插件时，唯一的要求是必须将其置于 _nvinfer_ 插件之后，以便在其中加入 AI 推理元数据。如前文所述，我们需要对探针[回调函数](https://en.wikipedia.org/wiki/Callback_(computer_programming))进行编程，以使此函数在车辆可能发生追尾时向我们发送信号。探针回调函数通常按样板帮助您遍历批量数据、帧和物体。如需有关如何实现回调函数的更多信息，请参阅 [GStreamer 探针文档](https://gstreamer.freedesktop.org/documentation/additional/design/probes.html)。

<p><img src='images/probe_boiler_plate.png' width=720></p>

我们希望为每帧生成一个包含 0 和 1 的列表，以表示其中是否出现追尾。因此，最终应该有与帧数一样多的列表编号。单个编号_不_能与检测到的每个物体相关联，因为这将导致多个编号与每帧相关联。示例输出如下：

<p><img src='images/sample_log.png' width=720></p>

**说明**：
<br>
4.1. 查看工作流架构。<br>
4.2. *只将* 单元中 `<FIXME>` 修改为正确的代码，并执行此单元，以定义构建和运行工作流的函数。<br>
4.3. *只将* 单元中 `<FIXME>` 修改为正确的代码，并执行此单元，以定义探针回调函数。<br>
4.4. 执行此单元以运行工作流。<br>

In [ ]:
# 4.2
#Import necessary libraries
import sys
import gi
gi.require_version('Gst', '1.0')
from gi.repository import GObject, Gst, GLib
from common.bus_call import bus_call
import pyds

def run(input_file_path):
    global inference_output
    inference_output=[]
    Gst.init(None)

    # Create element that will form a pipeline
    print("Creating Pipeline")
    pipeline=Gst.Pipeline()
    
    source=Gst.ElementFactory.make(<<<<FIXME>>>>, "file-source")
    source.set_property('location', <<<<FIXME>>>>)
    h264parser=Gst.ElementFactory.make(<<<<FIXME>>>>, "h264-parser")
    decoder=Gst.ElementFactory.make("nvv4l2decoder", "nvv4l2-decoder")
    
    streammux=Gst.ElementFactory.make(<<<<FIXME>>>>, "Stream-muxer")    
    streammux.set_property('width', <<<<FIXME>>>>)
    streammux.set_property('height', <<<<FIXME>>>>)
    streammux.set_property('batch-size', <<<<FIXME>>>>)
    
    pgie=Gst.ElementFactory.make(<<<<FIXME>>>>, "primary-inference")
    pgie.set_property('config-file-path', os.environ['SPEC_FILE'])
    
    nvvidconv1=Gst.ElementFactory.make("nvvideoconvert", "convertor")
    nvosd=Gst.ElementFactory.make(<<<<FIXME>>>>, "onscreendisplay")
    nvvidconv2=Gst.ElementFactory.make("nvvideoconvert", "convertor2")
    capsfilter=Gst.ElementFactory.make("capsfilter", "capsfilter")
    caps=Gst.Caps.from_string("video/x-raw, format=I420")
    capsfilter.set_property("caps", caps)
    
    encoder=Gst.ElementFactory.make("avenc_mpeg4", "encoder")
    encoder.set_property("bitrate", 2000000)
    
    sink=Gst.ElementFactory.make(<<<<FIXME>>>>, 'filesink')
    sink.set_property('location', 'output.mpeg4')
    sink.set_property("sync", 1)
    
    # Add the elements to the pipeline
    print("Adding elements to Pipeline")
    pipeline.add(source)
    pipeline.add(h264parser)
    pipeline.add(decoder)
    pipeline.add(streammux)
    pipeline.add(<<<<FIXME>>>>)
    pipeline.add(nvvidconv1)
    pipeline.add(nvosd)
    pipeline.add(nvvidconv2)
    pipeline.add(capsfilter)
    pipeline.add(encoder)
    pipeline.add(sink)

    # Link the elements together
    print("Linking elements in the Pipeline")
    source.link(h264parser)
    h264parser.link(decoder)
    decoder.get_static_pad('src').link(streammux.get_request_pad("sink_0"))
    streammux.link(<<<<FIXME>>>>)
    <<<<FIXME>>>>.link(nvvidconv1)
    nvvidconv1.link(<<<<FIXME>>>>)
    <<<<FIXME>>>>.link(nvvidconv2)
    nvvidconv2.link(capsfilter)
    capsfilter.link(encoder)
    encoder.link(sink)
    
    # Attach probe to OSD sink pad
    osdsinkpad=nvosd.get_static_pad("sink")
    <<<<FIXME>>>>.add_probe(Gst.PadProbeType.BUFFER, <<<<FIXME>>>>, 0)

    # Create an event loop and feed gstreamer bus mesages to it
    loop=GLib.MainLoop()
    bus=pipeline.get_bus()
    bus.add_signal_watch()
    bus.connect("message", bus_call, loop)
    
    # Start play back and listen to events
    print("Starting pipeline")
    
    pipeline.set_state(Gst.State.PLAYING)
    try:
        loop.run()
    except:
        pass
    
    pipeline.set_state(Gst.State.NULL)
    return inference_output

In [ ]:
# 4.3
# Define the Probe Function
def osd_sink_pad_buffer_probe(pad, info, u_data):
    gst_buffer=info.get_buffer()

    # Retrieve batch metadata from the gst_buffer
    batch_meta=pyds.gst_buffer_get_nvds_batch_meta(hash(gst_buffer))
    l_frame=batch_meta.frame_meta_list
    while l_frame is not None:
        
        # Initially set the tailgate indicator to False for each frame
        tailgate=False
        try:
            frame_meta=pyds.NvDsFrameMeta.cast(l_frame.data)
        except StopIteration:
            break
        frame_number=frame_meta.frame_num
        l_obj=frame_meta.obj_meta_list
        
        # Iterate through each object to check its dimension
        while l_obj is not None:
            try:
                obj_meta=pyds.NvDsObjectMeta.cast(l_obj.data)
                
                # If the object meet the criteria then set tailgate indicator to True
                obj_bottom=obj_meta.rect_params.top+obj_meta.rect_params.height
                if (<<<<FIXME>>>> > FRAME_WIDTH*.3) & (<<<<FIXME>>>> > FRAME_HEIGHT*.9): 
                    tailgate=True
                    
            except StopIteration:
                break
            try: 
                l_obj=l_obj.next
            except StopIteration:
                break
        
        print(f'Analyzing frame {frame_number}', end='\r')
        inference_output.append(str(int(<<<<FIXME>>>>)))
        try:
            l_frame=l_frame.next
        except StopIteration:
            break
    return Gst.PadProbeReturn.OK

In [ ]:
# 4.4
tailgate_log=run(input_file_path='/dli/task/data/assessment_stream.h264')

# DO NOT CHANGE BELOW
with open('/dli/task/my_assessment/answer_4.txt', 'w') as f: 
    f.write('\n'.join(tailgate_log))

## 第 5 步：分析结果 ##
最后，可以使用收集的日志分析驾驶行为。

**说明**：<br>
5.1. 执行此单元，以将追尾日志导入 Pandas DataFrame。<br>
5.2. 执行此单元以标出发生追尾的次数。<br>
5.3. 确保引用输出 `.mp4` 文件，并执行此单元，以查看在原始视频中绘制边界框的合成文件。<br>
5.4. 执行此单元，以计算该车辆追尾的平均时间。<br>
5.5. *只* 修改 `<FIXME>` ，并标记您的答案。<br>

In [ ]:
# 5.1
# DO NOT CHANGE THIS CELL
import pandas as pd

df=pd.read_csv('my_assessment/answer_4.txt', names=['inference'])
df.head()

In [ ]:
# 5.2
# DO NOT CHANGE THIS CELL
import matplotlib.pyplot as plt
import numpy as np

df.plot(kind='bar', figsize=(30, 5))
plt.xticks(np.arange(0, len(df)+1, FRAME_RATE), np.arange(0, len(df)/FRAME_RATE))
plt.show()

In [ ]:
# 5.3
!ffmpeg -i output.mpeg4 output_converted.mp4 \
        -y \
        -loglevel quiet

# DO NOT CHANGE BELOW
Video('output_converted.mp4', width=720)

In [ ]:
# 5.4
# DO NOT CHANGE THIS CELL
display(df['inference'].value_counts(normalize=True))

In [ ]:
# Question: How much time (without the percentage sign, e.g. 5.0) did the vehicle tailgate? 
Answer=<<<<FIXME>>>>

# EXAMPLE: 
# Answer='5.0'

# DO NOT CHANGE BELOW
!echo $Answer > my_assessment/answer_5.txt

## 对代码评分 ##
如果您已检查完这 5 个问题并确认工作流正常运行，请保存对 Notebook 的更改，并重新访问启动此交互环境所使用的网页。如下方屏幕截图所示，单击 "**ASSESS TASK**"（评估任务）按钮。这样，您可获得这部分实验的积分，为自己赢取整个课程的能力证书。

<p><img src='images/credit.png' width=1080></p>

### 额外练习。可视化帧 ###
我们在下面加入了一些实用函数，可帮助您可视化显示追尾行为的帧。

**说明**：<br>
B.1. 执行此单元以提取追尾帧。<br>
B.2. 执行此单元，以随机显示所选追尾帧。<br>

In [ ]:
# B.1
# DO NOT CHANGE THIS CELL
import cv2

!mkdir output_images
!rm -r output_images/*
input_video=cv2.VideoCapture('output_converted.mp4')
retVal, im=input_video.read()
frameCount=0
while retVal:
    if frameCount in df[df['inference']==1].index:
        cv2.imwrite("output_images/frame_%d.jpg" % frameCount, im)     # save frame as JPEG file      
    retVal, im=input_video.read()
    print(f'Read a new frame: {frameCount}', end='\r')
    frameCount+=1
input_video.release()

In [ ]:
# B.2
# DO NOT CHANGE THIS CELL
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
from matplotlib.pyplot import imshow
import numpy as np

def plot_random_samples(frames):
    sample_frames = np.random.choice(frames,size=8)
    fig=plt.figure(figsize=(30, 8))
    columns = 4
    rows = 2
    i = 1 
    for frame_num in sample_frames:
        # im = Image.open('{}/images/{}/{}.jpg'.format(config["Base_Dest_Folder"], config["Test_Video_ID"], box["frame_no"]))
        im = Image.open(f'output_images/frame_{frame_num}.jpg')
        fig.add_subplot(rows, columns, i)
        i += 1
        plt.imshow(np.asarray(im))
    plt.show()
    
plot_random_samples(df[df['inference']==1].index)

<a href="https://www.nvidia.com/dli"><img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/></a>